In [1]:
import json
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option("display.max_colwidth",20)
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

In [2]:
def load_data(files_to_load):
    # 2. Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    DF_dict = {}
    for file in files_to_load:
        # 4. Read in the raw wiki movie data as a Pandas DataFrame.
        if file[-3:] == "csv":
            DF_dict[file] = pd.read_csv(file)
            
        # 3. Open the read the Wikipedia data JSON file.
        elif file[-4:] == "json":
            DF_dict[file] = pd.read_json(file)
        else:
            raise ValueError
        
    # 5. Return the three DataFrames
    return DF_dict

In [3]:
# 6 Create the path to your file directory and variables for the three files. 
file_dir = '.'
# Wikipedia data
wiki_file = f'{file_dir}/wikipedia-movies.json'
# Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

# 7. Set the three variables in Step 6 equal to the function created in Step 1.
data_dict = load_data([wiki_file, kaggle_file, ratings_file])

wiki_movies_df = data_dict[wiki_file]
kaggle_metadata = data_dict[kaggle_file]
ratings = data_dict[ratings_file]

C:\Users\Pangloss\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:11: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  # This is added back by InteractiveShellApp.init_path()


In [4]:
#  Add the clean movie function that takes in the argument, "movie".
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    # combine alternate titles into one list
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune-Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles

    # merge column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')

    return movie

In [5]:
# 12. Add the parse_dollars function.
def parse_dollars(s):
    # if s is not a string, return NaN
    if type(s) != str:
        return np.nan

    # if input is of the form $###.# million
    if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " million"
        s = re.sub('\$|\s|[a-zA-Z]','', s)

        # convert to float and multiply by a million
        value = float(s) * 10**6

        # return value
        return value

    # if input is of the form $###.# billion
    elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " billion"
        s = re.sub('\$|\s|[a-zA-Z]','', s)

        # convert to float and multiply by a billion
        value = float(s) * 10**9

        # return value
        return value

    # if input is of the form $###,###,###
    elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):

        # remove dollar sign and commas
        s = re.sub('\$|,','', s)

        # convert to float
        value = float(s)

        # return value
        return value

    # otherwise, return NaN
    else:
        return np.nan

In [6]:
def load_data(files_to_load):
    DF_dict = {}
    for file_path in files_to_load:
        if file_path[-3:] == "csv":
            DF_dict[file_path] = pd.read_csv(file_path)
        elif file_path == f'{file_dir}/wikipedia-movies.json':
            file = open(file_path)
            wiki_movies = [movie for movie in json.load(file)
                   if ('Director' in movie or 'Directed by' in movie)
                       and 'imdb_link' in movie
                       and 'No. of episodes' not in movie]
            # 4. Write a list comprehension to iterate through the cleaned wiki movies list
            # and call the clean_movie function on each movie.
            wiki_movies = [clean_movie(movie) for movie in wiki_movies]
            DF_dict[file_path] = pd.DataFrame(wiki_movies)
        elif file[-4:] == "json":
            DF_dict[file_path] = pd.read_json(file_path)
        else:
            raise ValueError

    # 3. Write a list comprehension to filter out TV shows.
    
    return DF_dict

In [7]:
file_dir = '.'
# Wikipedia data
wiki_file = f'{file_dir}/wikipedia-movies.json'
# Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'
data_dict = load_data([wiki_file, kaggle_file, ratings_file])
# 5. Read in the cleaned movies list from Step 4 as a DataFrame.
wiki_movies_df = data_dict[wiki_file]
kaggle_metadata = data_dict[kaggle_file]
ratings = data_dict[ratings_file]

C:\Users\Pangloss\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  


In [8]:
try:
    wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
    print(len(wiki_movies_df))
    wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)

except (exception):
    print(exception)

7076


In [9]:
wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]

In [10]:
# 8. Create a variable that will hold the non-null values from the “Box office” column.
box_office = wiki_movies_df['Box office'].dropna()

# 9. Convert the box office data created in Step 8 to string values using the lambda and join functions.
box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)

# 10. Write a regular expression to match the six elements of "form_one" of the box office data.
form_one = r'\$\d+\.?\d*\s*[mb]illion'

# 11. Write a regular expression to match the three elements of "form_two" of the box office data.
form_two = r'\$\d{1,3}(?:,\d{3})+'

matches_form_one = box_office.str.contains(form_one, flags=re.IGNORECASE, na=False)
matches_form_two = box_office.str.contains(form_two, flags=re.IGNORECASE, na=False)
box_office[~matches_form_one & ~matches_form_two]

# 13. Clean the box office column in the wiki_movies_df DataFrame.
wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)

In [11]:
# 14. Clean the budget column in the wiki_movies_df DataFrame.
budget = wiki_movies_df['Budget'].dropna()
budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)
budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)
matches_form_one = budget.str.contains(form_one, flags=re.IGNORECASE, na=False)
matches_form_two = budget.str.contains(form_two, flags=re.IGNORECASE, na=False)
budget[~matches_form_one & ~matches_form_two]
wiki_movies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)

In [12]:
# 15. Clean the release date column in the wiki_movies_df DataFrame.
release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]?\d,\s\d{4}'
date_form_two = r'\d{4}.[01]\d.[0123]\d'
date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
date_form_four = r'\d{4}'
release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})', flags=re.IGNORECASE)
wiki_movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)

In [13]:
# 16. Clean the running time column in the wiki_movies_df DataFrame.
running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
running_time.str.contains(r'^\d*\s*minutes$', flags=re.IGNORECASE, na=False).sum()
running_time[running_time.str.contains(r'^\d*\s*minutes$', flags=re.IGNORECASE, na=False) != True]
running_time.str.contains(r'^\d*\s*m', flags=re.IGNORECASE, na=False).sum()
running_time[running_time.str.contains(r'^\d*\s*m', flags=re.IGNORECASE, na=False) != True]
running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')
running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)
wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)
wiki_movies_df.drop('Running time', axis=1, inplace=True)

In [14]:
# 2. Clean the Kaggle metadata.
kaggle_metadata = kaggle_metadata[kaggle_metadata['adult'] == 'False'].drop('adult',axis='columns')
kaggle_metadata['video'] = kaggle_metadata['video'] == 'True'
kaggle_metadata['budget'] = kaggle_metadata['budget'].astype(int)
kaggle_metadata['id'] = pd.to_numeric(kaggle_metadata['id'], errors='raise')
kaggle_metadata['popularity'] = pd.to_numeric(kaggle_metadata['popularity'], errors='raise')
kaggle_metadata['release_date'] = pd.to_datetime(kaggle_metadata['release_date'])

In [15]:
# 3. Merged the two DataFrames into the movies DataFrame.
movies_df = pd.merge(wiki_movies_df, kaggle_metadata, on='imdb_id', suffixes=['_wiki','_kaggle'])
movies_df

# 4. Drop unnecessary columns from the merged DataFrame.
movies_df.drop(columns=['title_wiki','release_date_wiki','Language','Production company(s)'], inplace=True)

# 5. Add in the function to fill in the missing Kaggle data.
def fill_missing_kaggle_data(df, kaggle_column, wiki_column):
    df[kaggle_column] = df.apply(
        lambda row: row[wiki_column] if row[kaggle_column] == 0 else row[kaggle_column]
        , axis=1)
    df.drop(columns=wiki_column, inplace=True)

# 6. Call the function in Step 5 with the DataFrame and columns as the arguments.
fill_missing_kaggle_data(movies_df, 'runtime', 'running_time')
fill_missing_kaggle_data(movies_df, 'budget_kaggle', 'budget_wiki')
fill_missing_kaggle_data(movies_df, 'revenue', 'box_office')
movies_df

# 7. Filter the movies DataFrame for specific columns.
movies_df = movies_df.loc[:, ['imdb_id','id','title_kaggle','original_title','tagline','belongs_to_collection','url','imdb_link',
                       'runtime','budget_kaggle','revenue','release_date_kaggle','popularity','vote_average','vote_count',
                       'genres','original_language','overview','spoken_languages','Country',
                       'production_companies','production_countries','Distributor',
                       'Producer(s)','Director','Starring','Cinematography','Editor(s)','Writer(s)','Composer(s)','Based on'
                      ]]

# 8. Rename the columns in the movies DataFrame.
movies_df.rename({'id':'kaggle_id',
                  'title_kaggle':'title',
                  'url':'wikipedia_url',
                  'budget_kaggle':'budget',
                  'release_date_kaggle':'release_date',
                  'Country':'country',
                  'Distributor':'distributor',
                  'Producer(s)':'producers',
                  'Director':'director',
                  'Starring':'starring',
                  'Cinematography':'cinematography',
                  'Editor(s)':'editors',
                  'Writer(s)':'writers',
                  'Composer(s)':'composers',
                  'Based on':'based_on'
                 }, axis='columns', inplace=True)

# 9. Transform and merge the ratings DataFrame.
rating_counts = ratings.groupby(['movieId','rating'], as_index=False).count() \
                .rename({'userId':'count'}, axis=1) \
                .pivot(index='movieId',columns='rating', values='count')
rating_counts.columns = ['rating_' + str(col) for col in rating_counts.columns]
movies_with_ratings_df = pd.merge(movies_df, rating_counts, left_on='kaggle_id', right_index=True, how='left')
movies_with_ratings_df[rating_counts.columns] = movies_with_ratings_df[rating_counts.columns].fillna(0)

In [16]:
wiki_movies_df

,url,year,imdb_link,title,Based on,Starring,Cinematography,Release date,Country,Language,...,Editor(s),Composer(s),Producer(s),Production company(s),Writer(s),imdb_id,box_office,budget,release_date,running_time
0,https://en.wikip...,1990,https://www.imdb...,The Adventures o...,"[Characters, by ...",[Andrew Dice Cla...,Oliver Wood,"[July 11, 1990, ...",United States,English,...,Michael Tronick,"[Cliff Eidelman,...","[Steve Perry, Jo...",Silver Pictures,"[David Arnott, J...",tt0098987,21400000.0,20000000.0,1990-07-11,102.0
1,https://en.wikip...,1990,https://www.imdb...,"After Dark, My S...","[the novel, Afte...","[Jason Patric, R...",Mark Plummer,"[May 17, 1990, (...",United States,English,...,Howard E. Smith,Maurice Jarre,"[Ric Kidney, Rob...",Avenue Pictures,"[James Foley, Ro...",tt0098994,2700000.0,6000000.0,1990-05-17,114.0
2,https://en.wikip...,1990,https://www.imdb...,Air America,"[Air America, by...","[Mel Gibson, Rob...",Roger Deakins,"[August 10, 1990...",United States,"[English, Lao]",...,"[John Bloom, Loi...",Charles Gross,Daniel Melnick,[Carolco Picture...,"[John Eskow, Ric...",tt0099005,57718089.0,35000000.0,1990-08-10,113.0
3,https://en.wikip...,1990,https://www.imdb...,Alice,NaN,"[Alec Baldwin, B...",Carlo Di Palma,"[December 25, 19...",United States,English,...,Susan E. Morse,NaN,Robert Greenhut,NaN,Woody Allen,tt0099012,7331647.0,12000000.0,1990-12-25,106.0
4,https://en.wikip...,1990,https://www.imdb...,Almost an Angel,NaN,"[Paul Hogan, Eli...",Russell Boyd,"December 19, 1990",US,English,...,David Stiven,Maurice Jarre,John Cornell,NaN,Paul Hogan,tt0099018,6939946.0,25000000.0,1990-12-19,95.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7071,https://en.wikip...,2018,https://www.imdb...,Holmes & Watson,[Sherlock Holmes...,"[Will Ferrell, J...",Oliver Wood,"[December 25, 20...",United States,English,...,Dean Zimmerman,Mark Mothersbaugh,"[Will Ferrell, A...",[Columbia Pictur...,Etan Cohen,tt1255919,41900000.0,42000000.0,2018-12-25,90.0
7072,https://en.wikip...,2018,https://www.imdb...,Vice,NaN,"[Christian Bale,...",Greig Fraser,"[December 11, 20...",United States,English,...,Hank Corwin,Nicholas Britell,"[Brad Pitt, Dede...",[Plan B Entertai...,Adam McKay,tt6266538,76100000.0,60000000.0,2018-12-11,132.0
7073,https://en.wikip...,2018,https://www.imdb...,On the Basis of Sex,NaN,"[Felicity Jones,...",Michael Grady,"[November 8, 201...",United States,English,...,Michelle Tesoro,Mychael Danna,Robert W. Cort,"[Focus Features,...",Daniel Stiepleman,tt4669788,38400000.0,20000000.0,2018-11-08,120.0
7074,https://en.wikip...,2018,https://www.imdb...,Destroyer,NaN,"[Nicole Kidman, ...",Julie Kirkwood,"[August 31, 2018...",United States,English,...,Plummy Tucker,Theodore Shapiro,"[Fred Berger, Ph...","[30West, Automat...","[Phil Hay, Matt ...",tt7137380,5500000.0,9000000.0,2018-08-31,123.0


In [17]:
movies_with_ratings_df

,imdb_id,kaggle_id,title,original_title,tagline,belongs_to_collection,wikipedia_url,imdb_link,runtime,budget,...,rating_0.5,rating_1.0,rating_1.5,rating_2.0,rating_2.5,rating_3.0,rating_3.5,rating_4.0,rating_4.5,rating_5.0
0,tt0098987,9548,The Adventures o...,The Adventures o...,Kojak. Columbo. ...,NaN,https://en.wikip...,https://www.imdb...,104.0,49000000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,tt0098994,25501,"After Dark, My S...","After Dark, My S...",All they risked ...,NaN,https://en.wikip...,https://www.imdb...,114.0,6000000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,tt0099005,11856,Air America,Air America,The few. The pro...,NaN,https://en.wikip...,https://www.imdb...,112.0,35000000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,tt0099012,8217,Alice,Alice,NaN,NaN,https://en.wikip...,https://www.imdb...,102.0,12000000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,tt0099018,25943,Almost an Angel,Almost an Angel,Who does he thin...,NaN,https://en.wikip...,https://www.imdb...,95.0,25000000.0,...,3.0,0.0,3.0,2.0,5.0,26.0,37.0,46.0,16.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6047,tt5639354,429191,A Fantastic Woman,Una mujer fantás...,NaN,NaN,https://en.wikip...,https://www.imdb...,104.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6048,tt5390066,390059,Permission,Permission,NaN,NaN,https://en.wikip...,https://www.imdb...,96.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6049,tt6304162,429174,Loveless,Нелюбовь,NaN,NaN,https://en.wikip...,https://www.imdb...,128.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6050,tt5795086,412302,Gemini,Gemini,NaN,NaN,https://en.wikip...,https://www.imdb...,92.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
movies_df

,imdb_id,kaggle_id,title,original_title,tagline,belongs_to_collection,wikipedia_url,imdb_link,runtime,budget,...,production_countries,distributor,producers,director,starring,cinematography,editors,writers,composers,based_on
0,tt0098987,9548,The Adventures o...,The Adventures o...,Kojak. Columbo. ...,NaN,https://en.wikip...,https://www.imdb...,104.0,49000000.0,...,[{'iso_3166_1': ...,20th Century Fox,"[Steve Perry, Jo...",Renny Harlin,[Andrew Dice Cla...,Oliver Wood,Michael Tronick,"[David Arnott, J...","[Cliff Eidelman,...","[Characters, by ..."
1,tt0098994,25501,"After Dark, My S...","After Dark, My S...",All they risked ...,NaN,https://en.wikip...,https://www.imdb...,114.0,6000000.0,...,[{'iso_3166_1': ...,Avenue Pictures,"[Ric Kidney, Rob...",James Foley,"[Jason Patric, R...",Mark Plummer,Howard E. Smith,"[James Foley, Ro...",Maurice Jarre,"[the novel, Afte..."
2,tt0099005,11856,Air America,Air America,The few. The pro...,NaN,https://en.wikip...,https://www.imdb...,112.0,35000000.0,...,[{'iso_3166_1': ...,TriStar Pictures,Daniel Melnick,Roger Spottiswoode,"[Mel Gibson, Rob...",Roger Deakins,"[John Bloom, Loi...","[John Eskow, Ric...",Charles Gross,"[Air America, by..."
3,tt0099012,8217,Alice,Alice,NaN,NaN,https://en.wikip...,https://www.imdb...,102.0,12000000.0,...,[{'iso_3166_1': ...,Orion Pictures,Robert Greenhut,Woody Allen,"[Alec Baldwin, B...",Carlo Di Palma,Susan E. Morse,Woody Allen,NaN,NaN
4,tt0099018,25943,Almost an Angel,Almost an Angel,Who does he thin...,NaN,https://en.wikip...,https://www.imdb...,95.0,25000000.0,...,[{'iso_3166_1': ...,Paramount Pictures,John Cornell,John Cornell,"[Paul Hogan, Eli...",Russell Boyd,David Stiven,Paul Hogan,Maurice Jarre,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6047,tt5639354,429191,A Fantastic Woman,Una mujer fantás...,NaN,NaN,https://en.wikip...,https://www.imdb...,104.0,NaN,...,[{'iso_3166_1': ...,[Participant Med...,[Juan de Dios La...,Sebastián Lelio,"[Daniela Vega, F...",Benjamín Echazar...,Soledad Salfate,[Sebastián Lelio...,Matthew Herbert,NaN
6048,tt5390066,390059,Permission,Permission,NaN,NaN,https://en.wikip...,https://www.imdb...,96.0,NaN,...,[{'iso_3166_1': ...,Good Deed Entert...,"[Brian Crano, Re...",Brian Crano,"[Rebecca Hall, D...",Adam Bricker,Matt Friedman,Brian Crano,NaN,NaN
6049,tt6304162,429174,Loveless,Нелюбовь,NaN,NaN,https://en.wikip...,https://www.imdb...,128.0,NaN,...,[{'iso_3166_1': ...,[Sony Pictures R...,[Alexander Rodny...,Andrey Zvyagintsev,"[Maryana Spivak,...",Mikhail Krichman,Anna Mass,"[Oleg Negin, And...",[Evgueni Galperi...,NaN
6050,tt5795086,412302,Gemini,Gemini,NaN,NaN,https://en.wikip...,https://www.imdb...,92.0,NaN,...,[{'iso_3166_1': ...,Neon,"[Mynette Louie, ...",Aaron Katz,"[Lola Kirke, Zoë...",Andrew Reed,Aaron Katz,Aaron Katz,Keegan DeWitt,NaN
